In [2]:
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:

# Load the dataset
data = pd.read_csv('fer2013.csv')

In [4]:
# Preprocess the data
def preprocess_data(data):
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(width, height)
        face = cv2.resize(face.astype('uint8'), (width, height))
        faces.append(face.astype('float32'))
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    faces /= 255.0
    emotions = pd.get_dummies(data['emotion']).values
    return faces, emotions

faces, emotions = preprocess_data(data)
X_train, X_val, y_train, y_val = train_test_split(faces, emotions, test_size=0.2, random_state=42)

model = Sequential()

In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
#MaxPool takes a set of pixels (e.g., 2x2 views) and returns the maximum of the set. 
# This is believed to have the effect of returning the features in the image that maximise
# the activation deeper in the network: i.e., maximise the predictability.

# AvgPool takes a set of pixels (e.g., 2x2 views) and returns the average of the set. This has the 
# effect of averaging over an image or feature set to reduce its computational size and complexity.

# I prefer AvgPool lower in the stack — closer to the image input layer — and MaxPool deeper in
# the stack — closer to the label output layer. The use and location of pooling layers are up 
# to the NN architect (you).

In [ ]:

# Fit the data generator on the training data
datagen.fit(X_train)

# First convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third convolutional layer
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Fourth convolutional layer
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the layers
model.add(Flatten())

# Fully connected layer
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # 7 classes of emotions

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])

In [ ]:
# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 64)        640       
                                                                 
 batch_normalization (Batch  (None, 46, 46, 64)        256       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 128)       73856     
                                                                 
 batch_normalization_1 (Bat  (None, 21, 21, 128)       512       
 chNormalization)                                                
                                                        

In [ ]:
# Learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_scheduler = LearningRateScheduler(scheduler)
# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Model checkpoint
model_checkpoint = ModelCheckpoint('model_one.h5', save_best_only=True, monitor='val_loss', mode='min')

# Combine all callbacks
callbacks = [lr_scheduler, early_stopping, model_checkpoint]

In [ ]:
# Training the model
batch_size = 64
epochs = 50

history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(X_train) // batch_size,
                    epochs=epochs,
                    validation_data=(X_val, y_val),
                    callbacks=callbacks,
                    verbose=2)

Epoch 1/50
448/448 - 234s - loss: 1.9073 - accuracy: 0.2281 - auc: 0.6271 - val_loss: 1.8333 - val_accuracy: 0.2459 - val_auc: 0.6415 - lr: 0.0010 - 234s/epoch - 523ms/step
Epoch 2/50


C:\Users\USER\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


448/448 - 214s - loss: 1.8002 - accuracy: 0.2571 - auc: 0.6671 - val_loss: 1.8459 - val_accuracy: 0.2474 - val_auc: 0.6554 - lr: 0.0010 - 214s/epoch - 479ms/step
Epoch 3/50
448/448 - 207s - loss: 1.7249 - accuracy: 0.2962 - auc: 0.7049 - val_loss: 1.7996 - val_accuracy: 0.1853 - val_auc: 0.6613 - lr: 0.0010 - 207s/epoch - 463ms/step
Epoch 4/50
448/448 - 210s - loss: 1.6417 - accuracy: 0.3427 - auc: 0.7420 - val_loss: 1.5525 - val_accuracy: 0.4055 - val_auc: 0.7960 - lr: 0.0010 - 210s/epoch - 468ms/step
Epoch 5/50
448/448 - 211s - loss: 1.5697 - accuracy: 0.3855 - auc: 0.7726 - val_loss: 1.6190 - val_accuracy: 0.3795 - val_auc: 0.7611 - lr: 0.0010 - 211s/epoch - 471ms/step
Epoch 6/50
448/448 - 205s - loss: 1.5217 - accuracy: 0.4079 - auc: 0.7902 - val_loss: 1.5099 - val_accuracy: 0.4146 - val_auc: 0.8022 - lr: 0.0010 - 205s/epoch - 457ms/step
Epoch 7/50
448/448 - 202s - loss: 1.4696 - accuracy: 0.4344 - auc: 0.8071 - val_loss: 1.3447 - val_accuracy: 0.4649 - val_auc: 0.8386 - lr: 0.0010

In [ ]:
# Evaluate the model
loss, accuracy, AUC = model.evaluate(X_val, y_val, verbose=0)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

# Function to predict and display emotion
def predict_emotion(face):
    face = cv2.resize(face, (48, 48))
    face = np.expand_dims(face, axis=0)
    face = np.expand_dims(face, axis=-1)
    face = face / 255.0
    emotion_label = np.argmax(model.predict(face))
    return emotion_label

In [ ]:
# Example prediction
test_face = X_val[5]
predicted_emotion = predict_emotion(test_face)
emotion_map = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

plt.imshow(test_face.squeeze(), cmap='gray')
plt.title(f'Predicted Emotion: {emotion_map[predicted_emotion]}')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc, UAC = model.evaluate(X_val,  y_val, verbose=2)


In [ ]:
print(test_acc)